In [24]:
import pickle
from typing import Dict, List, Any, Union
import numpy as np
# Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.python.framework.random_seed import set_random_seed
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [5]:
def load_data() -> Dict[str, Union[List[Any], int]]:
    path = "keras-data.pickle"
    with open(file=path, mode="rb") as file:
        data = pickle.load(file)

    return data

In [6]:
def preprocess_data(data: Dict[str, Union[List[Any], int]]) -> Dict[str, Union[List[Any], np.ndarray, int]]:
    """
    Preprocesses the data dictionary. Both the training-data and the test-data must be padded
    to the same length; play around with the maxlen parameter to trade off speed and accuracy.
    """
    maxlen = data["max_length"]//16
    data["x_train"] = pad_sequences(data['x_train'], maxlen=maxlen)
    data["y_train"] = np.asarray(data['y_train'])
    data["x_test"] = pad_sequences(data['x_test'], maxlen=maxlen)
    data["y_test"] = np.asarray(data['y_test'])

    return data

In [22]:
def train_model(data: Dict[str, Union[List[Any], np.ndarray, int]], model_type="feedforward") -> float:
    """
    Build a neural network of type model_type and train the model on the data.
    Evaluate the accuracy of the model on test data.

    :param data: The dataset dictionary to train neural network on
    :param model_type: The model to be trained, either "feedforward" for feedforward network
                        or "recurrent" for recurrent network
    :return: The accuracy of the model on test data
    """
    # Configuration options
    #see the number of labels
    labels = set(data["y_train"])
    print(labels, len(labels))

    # Configuration options
    feature_vector_length = data["x_train"].shape[1] #how many features
    num_classes = len(labels) #how many classes
    y_train_cat = to_categorical(data["y_train"], num_classes)
    y_test_cat = to_categorical(data["y_test"], num_classes)
    vocab_size = data["vocab_size"]
    maxlen = data["max_length"]//16
    if(model_type == "feedforward"):
        np.random.seed(123)
        set_random_seed(2)

        #model = Sequential() #we first define how the "model" looks like
        #model.add(Dense(input_dim = feature_vector_length, units=feature_vector_length , activation='relu')) #input layer
        #model.add(Dense(num_classes, activation='softmax')) #output layer
        #print(model.summary())
        #plot_model(model, show_shapes= True)
        
        model = Sequential()
        model.add(Embedding(input_dim=vocab_size, output_dim=32, input_length=feature_vector_length))
        model.add(Flatten())
        model.add(Dense(input_dim = feature_vector_length, units=feature_vector_length , activation='relu')) #input layer
        model.add(Dense(num_classes, activation='softmax')) #output layer
        
        
        #model = tf.keras.Sequential([
        #tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=32, input_length=input_shape),
        #tf.keras.layers.Flatten(),
        #tf.keras.layers.Dense(128, activation='relu'),
        #tf.keras.layers.Dense(10, activation='softmax')])
        
        model.compile(loss='categorical_crossentropy', #loss metric
        optimizer='sgd',  #optimizer
        metrics=['accuracy']) #displayed metric


        #fit
        history = model.fit(data["x_train"], y_train_cat, epochs=15, batch_size=32, verbose=1, validation_split=0.1)
        
        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        plt.show()

        # summarize history for accuracy
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        plt.show()
        
        #test
        
        test_results = model.evaluate(data["x_test"], y_test_cat, verbose=1)
        print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
        return test_results[1]
        
    elif(model_type == "recurrent"):
        np.random.seed(123)
        set_random_seed(2)
        
        model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=32, input_length=maxlen),
        tf.keras.layers.LSTM(66, dropout=0.2, recurrent_dropout=0.2),
        tf.keras.layers.Dense(num_classes, activation='softmax')
        ])
        
        model.compile(loss='categorical_crossentropy', #loss metric
        optimizer='sgd',  #optimizer
        metrics=['accuracy']) #displayed metric


        #fit
        history = model.fit(data["x_train"], y_train_cat, epochs=5, batch_size=32, validation_split=0.2)
        
        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        plt.show()

        # summarize history for accuracy
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        plt.show()
        
        #test
        test_results = model.evaluate(data["x_test"], y_test_cat, verbose=1)
        print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
        return test_results[1]

    # TODO build the model given model_type, train it on (data["x_train"], data["y_train"])
    #  and evaluate its accuracy on (data["x_test"], data["y_test"]). Return the accuracy

In [23]:
def main() -> None:
    print("1. Loading data...")
    keras_data = load_data()
    print("2. Preprocessing data...")
    keras_data = preprocess_data(keras_data)
    print("3. Training feedforward neural network...")
    fnn_test_accuracy = train_model(keras_data, model_type="feedforward")
    #print('Model: Feedforward NN.\n'
    #      f'Test accuracy: {fnn_test_accuracy:.3f}')
    print("4. Training recurrent neural network...")
    rnn_test_accuracy = train_model(keras_data, model_type="recurrent")
    print('Model: Recurrent NN.\n'
          f'Test accuracy: {rnn_test_accuracy:.3f}')



if __name__ == '__main__':
    main()


1. Loading data...
2. Preprocessing data...
3. Training feedforward neural network...
{0, 1} 2


ValueError: Unknown optimizer: 'AD'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.